<a href="https://colab.research.google.com/github/JeswinJestin/GenAI_PPT_Generator/blob/main/PPT_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install google-generativeai python-pptx Pillow requests python-dotenv

In [40]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import io
import json
from PIL import Image
import requests

# Load environment variables from a .env file if it exists
load_dotenv()

False

In [41]:
import os
import json
import requests
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from PIL import Image, ImageDraw
import google.generativeai as genai
from dotenv import load_dotenv # Ensure dotenv is imported here too

# Load environment variables if not already loaded
load_dotenv()


class PPTGenerator:
    def __init__(self, api_key=None):
        # Prioritize explicitly passed API key, then environment variable
        self.api_key = api_key if api_key else os.getenv("GEMINI_API_KEY")
        if not self.api_key:
            raise ValueError("Gemini API Key not provided. Please set GEMINI_API_KEY in your environment or pass it to the constructor.")

        genai.configure(api_key=self.api_key)
        # Consider adding a check here to see if the API key is valid
        # by making a small test call.

        self.model = genai.GenerativeModel('gemini-2.5-pro')

        self.presentation = Presentation()
        # Load Pexels API key from environment variables
        self.pexels_api_key = os.getenv("PEXELS_API_KEY")
        if not self.pexels_api_key:
             print("⚠️ Pexels API Key not found in environment variables. Image downloads may fail.")


    def generate_content_outline(self, topic, num_slides=5):
        """Generate advanced content outline for a PowerPoint presentation using Gemini"""
        prompt = f"""
        You are an expert content creator and instructional designer.
        Create a comprehensive and engaging outline for a PowerPoint presentation on "{topic}" with {num_slides} slides.

        Guidelines:
        1. Ensure each slide has a clear purpose and logical flow from introduction to conclusion.
        2. Include slide types as: "title", "content", "image", or "conclusion".
        3. For content slides, provide 3-5 bullet points that are concise, actionable, and easy to understand.
        4. Suggest a relevant image idea for image slides.
        5. Provide short speaker notes for each slide (1-2 sentences).
        6. Suggest a simple design hint (color theme, layout style, or font tone).
        7. Return the response strictly as a valid JSON array with this structure:
           [
               {{
                   "title": "Slide Title",
                   "content": "Main content points as bullet points or description for image slide",
                   "slide_type": "title|content|image|conclusion",
                   "notes": "Speaker notes",
                   "design_hint": "Design suggestion"
               }}
           ]
        """

        try:
            response = self.model.generate_content(prompt)
            content = response.text.strip()

            # Handle markdown wrapping and direct list output
            if isinstance(response.candidates[0].content.parts[0].text, str):
                content = response.text.strip()
                if "```json" in content:
                    content = content.split("```json")[1].split("```")[0].strip()
                elif "```" in content:
                    # Assuming the raw JSON is sometimes returned without the json identifier
                    content = content.split("```")[1].strip()
                # Attempt to parse the JSON string
                return json.loads(content)
            elif isinstance(response.candidates[0].content.parts[0].text, list):
                # If the API returns a list directly, return it
                return response.candidates[0].content.parts[0].text
            else:
                # Handle unexpected response types
                print(f"⚠️ Unexpected API response format: {type(response.candidates[0].content.parts[0].text)}")
                return self._get_fallback_outline(topic, num_slides)


        except json.JSONDecodeError:
            print(f"⚠️ JSON Decode Error: Could not parse content as JSON. Content: {content[:500]}...") # Print a snippet of content
            return self._get_fallback_outline(topic, num_slides)
        except Exception as e:
            print(f"⚠️ Error generating content outline: {e}")
            return self._get_fallback_outline(topic, num_slides)


    def _get_fallback_outline(self, topic, num_slides):
        """Fallback outline if Gemini fails"""
        print("Using fallback outline.")
        # Generate a simple fallback outline based on the requested number of slides
        fallback = []
        if num_slides > 0:
            fallback.append({
                "title": f"Introduction to {topic}",
                "content": "• Overview of the topic\n• Key objectives\n• What to expect",
                "slide_type": "title",
                "notes": "Introduce the topic and set expectations.",
                "design_hint": "Clean white background with bold blue title"
            })
        for i in range(1, num_slides - 1):
             fallback.append({
                "title": f"{topic} - Key Point {i}",
                "content": f"• Detail 1 for key point {i}\n• Detail 2 for key point {i}",
                "slide_type": "content",
                "notes": f"Discuss key point {i}.",
                "design_hint": f"Layout variation {i % 2 + 1}"
            })
        if num_slides > 1:
             fallback.append({
                "title": "Conclusion",
                "content": "• Summary of key points\n• Final thoughts\n• Next steps",
                "slide_type": "conclusion",
                "notes": "Wrap up and leave audience with takeaways.",
                "design_hint": "Bold ending slide with contrasting colors"
            })
        # Ensure the fallback outline has the correct number of slides
        while len(fallback) < num_slides:
             fallback.append({
                "title": f"{topic} - Additional Point {len(fallback)}",
                "content": f"• Additional detail for point {len(fallback)}",
                "slide_type": "content",
                "notes": f"Additional point {len(fallback)}.",
                "design_hint": "Simple layout"
            })
        return fallback[:num_slides] # Return exactly the requested number of slides


    def generate_image_description(self, slide_content):
        """Generate image description for slides using Gemini"""
        prompt = f"""
        Based on this slide content, suggest a relevant image description that would enhance the presentation:

        {slide_content}

        Return only a brief, descriptive phrase suitable for image search (max 10 words).
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            print(f"⚠️ Error generating image description: {e}")
            return "professional presentation"

    def download_image(self, query, save_path="temp_image.jpg"):
        """Download a relevant image from Pexels based on the query"""
        if not self.pexels_api_key:
            print("⚠️ Pexels API key is not set. Skipping image download.")
            return self._create_fallback_image(save_path, "Pexels API Key Missing")

        try:
            url = "https://api.pexels.com/v1/search"
            headers = {"Authorization": self.pexels_api_key}
            params = {"query": query, "per_page": 1, "orientation": "landscape"}

            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()

            data = response.json()
            if not data.get("photos"):
                raise ValueError(f"No images found for query: '{query}'.")

            image_url = data["photos"][0]["src"]["original"]

            img_response = requests.get(image_url)
            img_response.raise_for_status()

            with open(save_path, "wb") as f:
                f.write(img_response.content)

            return save_path

        except requests.exceptions.RequestException as e:
            print(f"⚠️ Error downloading image from Pexels: {e}")
            return self._create_fallback_image(save_path, f"Image Download Failed: {e}")
        except ValueError as e:
             print(f"⚠️ Error in Pexels API response: {e}")
             return self._create_fallback_image(save_path, f"No Image Found: {e}")
        except Exception as e:
            print(f"⚠️ An unexpected error occurred during image download: {e}")
            return self._create_fallback_image(save_path, f"Image Error: {e}")

    def _create_fallback_image(self, save_path, text="No Image Available"):
        """Creates a simple fallback image with text."""
        try:
            img = Image.new("RGB", (800, 600), color="#DDDDDD") # Use a neutral color
            draw = ImageDraw.Draw(img)
            # Consider using a better font and centering the text
            # Simple text wrapping might be needed for long messages
            draw.text((200, 280), text, fill="#333333")
            img.save(save_path)
            return save_path
        except Exception as e:
            print(f"⚠️ Error creating fallback image: {e}")
            return None


    def create_title_slide(self, title, subtitle=""):
        slide_layout = self.presentation.slide_layouts[0]
        slide = self.presentation.slides.add_slide(slide_layout)

        title_shape = slide.shapes.title
        title_shape.text = title
        # Improve font handling and styling based on design hints later
        title_shape.text_frame.paragraphs[0].font.size = Pt(44)
        title_shape.text_frame.paragraphs[0].font.bold = True
        title_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        if subtitle:
            subtitle_shape = slide.placeholders[1]
            subtitle_shape.text = subtitle
            subtitle_shape.text_frame.paragraphs[0].font.size = Pt(24)
            subtitle_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        return slide

    def create_content_slide(self, title, content, notes="", include_image=False):
        slide_layout = self.presentation.slide_layouts[1]
        slide = self.presentation.slides.add_slide(slide_layout)

        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(36)
        title_shape.text_frame.paragraphs[0].font.bold = True

        content_shape = slide.placeholders[1]
        # Handle potential non-string content if API output format changes unexpectedly
        content_shape.text = str(content) if content is not None else ""
        text_frame = content_shape.text_frame
        for paragraph in text_frame.paragraphs:
            paragraph.font.size = Pt(18)
            paragraph.font.color.rgb = RGBColor(51, 51, 51)

        # Add speaker notes
        if notes:
            notes_slide = slide.notes_slide
            notes_slide.notes_text_frame.text = notes

        if include_image:
            try:
                image_desc = self.generate_image_description(content)
                if image_desc: # Only attempt download if description is generated
                    image_path = self.download_image(image_desc)
                    if image_path and os.path.exists(image_path):
                        # Consider better image placement and sizing logic
                        slide.shapes.add_picture(image_path, Inches(6), Inches(2), height=Inches(4))
                        os.remove(image_path)
            except Exception as e:
                print(f"⚠️ Error adding image to content slide: {e}")


        return slide

    def create_image_slide(self, title, content, image_query, notes=""):
        slide_layout = self.presentation.slide_layouts[8]
        slide = self.presentation.slides.add_slide(slide_layout)

        # Using text boxes for more flexible positioning
        title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1))
        title_frame = title_box.text_frame
        title_frame.text = title
        title_frame.paragraphs[0].font.size = Pt(36)
        title_frame.paragraphs[0].font.bold = True
        title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        content_box = slide.shapes.add_textbox(Inches(0.5), Inches(2), Inches(4), Inches(5))
        content_frame = content_box.text_frame
        # Handle potential non-string content
        content_frame.text = str(content) if content is not None else ""
        for paragraph in content_frame.paragraphs:
            paragraph.font.size = Pt(18)
            paragraph.font.color.rgb = RGBColor(51, 51, 51)

        if notes:
            notes_slide = slide.notes_slide
            notes_slide.notes_text_frame.text = notes

        try:
            if image_query: # Only attempt download if query is provided
                image_path = self.download_image(image_query)
                if image_path and os.path.exists(image_path):
                    # Consider better image placement and sizing for image slides
                    slide.shapes.add_picture(image_path, Inches(5), Inches(2), height=Inches(5))
                    os.remove(image_path)
        except Exception as e:
            print(f"⚠️ Error adding image to image slide: {e}")

        return slide

    def generate_presentation(self, topic, num_slides=5, output_path="generated_presentation.pptx"):
        print(f"🚀 Generating presentation on: {topic}")

        outline = self.generate_content_outline(topic, num_slides)

        if not outline:
            print("⚠️ Could not generate presentation outline. Aborting.")
            return None

        for i, slide_data in enumerate(outline):
            # Ensure slide_data is a dictionary and has necessary keys
            if not isinstance(slide_data, dict):
                 print(f"⚠️ Skipping invalid slide data at index {i}: {slide_data}")
                 continue

            title = slide_data.get("title", f"Slide {i+1}") # Provide a default title
            content = slide_data.get("content", "")
            slide_type = slide_data.get("slide_type", "content").lower() # Default to content, handle case
            notes = slide_data.get("notes", "")
            design_hint = slide_data.get("design_hint", "")

            print(f"📝 Creating slide {i+1}: {title} | Type: {slide_type} | Design: {design_hint}")

            try:
                if i == 0 or slide_type == "title":
                    self.create_title_slide(title, "Generated by Gemini AI")
                elif slide_type == "image":
                    # Use the content as a basis for image query if not explicitly provided in outline (though prompt asks for it)
                    image_query = slide_data.get("image_query", self.generate_image_description(content))
                    self.create_image_slide(title, content, image_query, notes)
                elif slide_type == "conclusion":
                     # Could create a specific conclusion slide layout
                     self.create_content_slide(title, content, notes, include_image=False) # Conclusion usually doesn't need image
                elif slide_type == "content":
                    # Decide whether to include an image based on index or other criteria
                    include_image = (i % 2 == 1) # Example: include image on every other content slide
                    self.create_content_slide(title, content, notes, include_image)
                else:
                    print(f"⚠️ Unknown slide type '{slide_type}' for slide {i+1}. Creating as content slide.")
                    self.create_content_slide(title, content, notes, include_image=False) # Fallback for unknown types
            except Exception as e:
                 print(f"❌ Error creating slide {i+1} ('{title}'): {e}")
                 # Optionally, add a placeholder slide to indicate an error occurred
                 try:
                     slide_layout = self.presentation.slide_layouts[5] # Blank slide layout
                     slide = self.presentation.slides.add_slide(slide_layout)
                     title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1))
                     title_frame = title_box.text_frame
                     title_frame.text = f"Error creating slide {i+1}"
                     title_frame.paragraphs[0].font.size = Pt(36)
                     content_box = slide.shapes.add_textbox(Inches(0.5), Inches(2), Inches(9), Inches(5))
                     content_frame = content_box.text_frame
                     content_frame.text = f"An error occurred while generating this slide: {e}"
                 except Exception as slide_error:
                      print(f"❌ Failed to add error placeholder slide: {slide_error}")


        try:
            self.presentation.save(output_path)
            print(f"✅ Presentation saved as: {output_path}")
            return output_path
        except Exception as e:
            print(f"❌ Error saving presentation: {e}")
            return None


print("✅ PPTGenerator class defined successfully with enhanced error handling and structure!")

✅ PPTGenerator class defined successfully with enhanced error handling and structure!


## Setup Gemini API Key
#### You need to set up your Gemini API key. You can either:

1. Create a .env file with GEMINI_API_KEY=your_api_key_here
2. Set it directly in the code below

In [42]:
# Check if API key is loaded from environment or set
api_key = os.getenv("GEMINI_API_KEY") # Try to get from environment first
if api_key:
    print("✅ Gemini API key is configured (loaded from environment)")
else:
    # Fallback to hardcoded key ONLY for demonstration if environment variable is not set
    # In a real application, you would NOT hardcode keys like this.
    # api_key = "YOUR_HARDCODED_API_KEY_HERE" # Uncomment and replace ONLY for testing without .env
    if api_key: # Check again if hardcoded key was set
         print("⚠️ Using hardcoded Gemini API key. Consider using a .env file for security.")
    else:
         print("❌ Gemini API key is not configured. Please set GEMINI_API_KEY in your .env file or environment variables.")

# Check for Pexels API key as well
pexels_api_key = os.getenv("PEXELS_API_KEY")
if pexels_api_key:
    print("✅ Pexels API key is configured (loaded from environment)")
else:
    print("⚠️ Pexels API key is not configured. Image downloads may fail. Set PEXELS_API_KEY in your .env file or environment variables.")

❌ Gemini API key is not configured. Please set GEMINI_API_KEY in your .env file or environment variables.
⚠️ Pexels API key is not configured. Image downloads may fail. Set PEXELS_API_KEY in your .env file or environment variables.


In [43]:
# Initialize the generator
# Pass the loaded api_key (which could be None if not found)
try:
    generator = PPTGenerator(api_key=api_key)
    print("✅ PPT Generator initialized successfully!")
except ValueError as e:
    print(f"❌ Error initializing PPTGenerator: {e}")
    print("Please set your GEMINI_API_KEY first.")

❌ Error initializing PPTGenerator: Gemini API Key not provided. Please set GEMINI_API_KEY in your environment or pass it to the constructor.
Please set your GEMINI_API_KEY first.


# **Generate a Presentation**
Now let's create a presentation! You can change the topic and number of slides as needed.

In [44]:
# Initialize the generator
# Pass the loaded api_key (which could be None if not found)
try:
    generator = PPTGenerator(api_key=api_key)
    print("✅ PPT Generator initialized successfully!")
except ValueError as e:
    print(f"❌ Error initializing PPTGenerator: {e}")
    print("Please set your GEMINI_API_KEY first.")

❌ Error initializing PPTGenerator: Gemini API Key not provided. Please set GEMINI_API_KEY in your environment or pass it to the constructor.
Please set your GEMINI_API_KEY first.


## Using Colab Secrets Manager for API Keys

Instead of storing API keys directly in `.env` files or code, the most secure way in Colab is to use the built-in Secrets Manager.

1.  Click the "🔑" icon in the left sidebar.
2.  Click "Add new secret".
3.  Set the **Name** to `GOOGLE_API_KEY`.
4.  Paste your API key into the **Value** field.
5.  Click "Save secret".
6.  Ensure the secret is enabled for this notebook.

Then, you can access the key in your code:

In [45]:
from google.colab import userdata
import os

# Get the API key from Colab Secrets
# Note: The name here is 'GOOGLE_API_KEY' as recommended by Colab documentation
api_key = userdata.get('GOOGLE_API_KEY')

# You might also want to get your Pexels API key if you have one
# pexels_api_key = userdata.get('PEXELS_API_KEY')

if api_key:
    print("✅ Gemini API key loaded from Colab Secrets.")
    # Now you can initialize your PPTGenerator with this key
    # try:
    #     generator = PPTGenerator(api_key=api_key)
    #     print("✅ PPT Generator initialized successfully!")
    # except ValueError as e:
    #     print(f"❌ Error initializing PPTGenerator: {e}")
    #     print("Please ensure your API key is correct.")
else:
    print("❌ GOOGLE_API_KEY not found in Colab Secrets. Please add it using the 🔑 icon.")

# If you are still using dotenv for other variables, keep this line
# load_dotenv() # Assuming load_dotenv is imported earlier

✅ Gemini API key loaded from Colab Secrets.


In [46]:
# Generate a presentation
topic = "Artificial Intelligence in Healthcare"  # Change this to your desired topic
num_slides = 6  # Change this to your desired number of slides

try:
    output_file = generator.generate_presentation(topic, num_slides, "ai_healthcare_presentation.pptx")
except Exception as e:
    print(e)

🚀 Generating presentation on: Artificial Intelligence in Healthcare


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1421.45ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 647.19ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 857.89ms


⚠️ Error generating content outline: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 50.074196362s.
Using fallback outline.
📝 Creating slide 1: Introduction to Artificial Intelligence in Healthcare | Type: title | Design: Clean white background with bold blue title
📝 Creating slide 2: Artificial Intelligence in Healthcare - Key Point 1 | Type: content | Design: Layout variation 2


⚠️ Error generating image description: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 49.502448319s.
📝 Creating slide 3: Artificial Intelligence in Healthcare - Key Point 2 | Type: content | Design: Layout variation 1
📝 Creating slide 4: Artificial Intelligence in Healthcare - Key Point 3 | Type: content | Design: Layout variation 2
📝 Creating slide 5: Artificial Intelligence in Healthcare - Key Point 4 | Type: content | Design: Layout variation 1
📝 Creating slide 6: Conclusion | Type: conclusion | Design: Bold ending slide with contrasting colors
✅ Presentation saved as: ai_healthcare_presentation.pptx
